# Preprocessing of PTB-XL-700 Dataset

In [1]:
import pandas as pd
import numpy as np
import wfdb
import ast
import os
from pathlib import Path
import torch
from typing import Optional
from scipy.stats import zscore

In [2]:
import plotly.graph_objects as go
import plotly.io as pio

# -------------------- DEFINE CUSTOM TEMPLATE -------------------- #
pio.templates['draft'] = go.layout.Template(layout=dict(
    margin=dict(l=50, r=50, b=50, t=50),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
)
))
pio.templates.default = "plotly_white+draft"

In [3]:
temp_dir = Path("ptb-xl-2s-univar")
os.makedirs(temp_dir, exist_ok=True)

get_subset = True
get_univar = True

## Pre-processing of the PTB-XL

### Load WFDB dataset

In [4]:
def load_raw_data(file_names: pd.Series, path):
    data = [wfdb.rdsamp(path+'/'+f, return_res=32) for f in file_names]
    data = np.array([signal for signal, meta in data])
    return data

def aggregate_diagnostic(y_dic, agg_df):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

In [5]:
ptbxl_path = '../dataset/ptb-xl-1.0.3 (500hz)'
sampling_rate=500

In [6]:
# load the dataset annotation and convert `scp_codes` values to list
Y = pd.read_csv(ptbxl_path + '/ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

records_path = Y['filename_hr'].copy() # the records path

# load the scp codes information
agg_df = pd.read_csv(ptbxl_path + '/scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

# aggregate the label 'diagnostic'
Y['label'] = Y.scp_codes.apply(aggregate_diagnostic, agg_df=agg_df)

# drop non relevant information
Y = Y.reset_index()
Y = Y[['ecg_id', 'strat_fold', 'label']].copy()

Y.head()

,ecg_id,strat_fold,label
0,1,3,[NORM]
1,2,2,[NORM]
2,3,5,[NORM]
3,4,3,[NORM]
4,5,4,[NORM]


In [7]:
# Load raw signal data
X = load_raw_data(records_path, ptbxl_path)
X = np.moveaxis(X, 1, 2)

print(X.shape, X.dtype)

(21799, 12, 5000) float32


### Split the data into (train, val, and test)

In [8]:
def one_hot_encoding(df: pd.DataFrame, 
                     list_label: list[str], 
                     label_col: Optional[str] = "label") -> np.ndarray:
    '''Convert the label (list of classes) into one-hot encoding for multilabel classification task.
    
    Args:
        df (pd.DataFrame): DataFrame with the labels.
        list_label (list[str]): List with all possibles classes.
        label_col (str, optional): Column to be encoded. Default "label".
    
    Returns:
        One-hot encoded labels (np.ndarray) with dimensions - len(`df`) x len(`list_label`).
    '''
    for label in list_label:

        # check if the label is at samples' list of labels
        df[label] = df[label_col].apply(lambda x: int(label in x)).copy()

    # extract the labels columns to numpy
    coded_labels = df[list_label].to_numpy()

    return coded_labels

In [9]:
# unique classes combinations
combinations = Y.label.value_counts().index.to_list()

# concat list of classes and get unique values
list_label = ['CD', 'MI', 'HYP', 'STTC', 'NORM']
print(list_label)

['CD', 'MI', 'HYP', 'STTC', 'NORM']


In [10]:
# SPLIT THE DATASET INTO TRAIN, VALIDATION AND TEST
# -------------------------------------------------

# test [fold 10]
x_test = X[np.where(Y.strat_fold == 10)]
y_test_df = Y[Y.strat_fold == 10].copy()
y_test = one_hot_encoding(y_test_df, list_label)

# Train and evaluation folds
x_temp = X[np.where(Y.strat_fold != 10)]
y_temp = Y[Y.strat_fold != 10]

# valutation [fold 9]
x_val = x_temp[np.where(y_temp.strat_fold == 9)]
y_val_df = y_temp[y_temp.strat_fold == 9].copy()
y_val = one_hot_encoding(y_val_df, list_label)

# train [fold 1-8]
x_train = x_temp[np.where(y_temp.strat_fold != 9)]
y_train_df = y_temp[y_temp.strat_fold != 9].copy()
y_train = one_hot_encoding(y_train_df, list_label)

#### Get subset PTB-XL

In [11]:
x_train.shape, x_val.shape, x_test.shape

((17418, 12, 5000), (2183, 12, 5000), (2198, 12, 5000))

In [13]:
if get_subset:
    np.random.seed(123)

    train_idx = np.random.choice(len(x_train), 532, replace=False)
    x_train = x_train[train_idx]
    y_train = y_train[train_idx]
    y_train_df = y_train_df.iloc[train_idx]

    val_idx = np.random.choice(len(x_val), 84, replace=False)
    x_val = x_val[val_idx]
    y_val = y_val[val_idx]
    y_val_df = y_val_df.iloc[val_idx]

    test_idx = np.random.choice(len(x_test), 84, replace=False)
    x_test = x_test[test_idx]
    y_test = y_test[test_idx]
    y_test_df = y_test_df.iloc[test_idx]

In [14]:
# torch.save({'samples': torch.from_numpy(x_train), 'labels': y_train}, 
#             f=temp_dir / "train_full.pt")

# torch.save({'samples': torch.from_numpy(x_val), 'labels': y_val}, 
#             f=temp_dir / "val_full.pt")

# torch.save({'samples': torch.from_numpy(x_test), 'labels': y_test}, 
#             f=temp_dir / "test_full.pt")

### Split each record into 5 segments of 2 seconds

In [15]:
import zipfile
import shutil
import json

In [16]:
def segment_samples(data: np.ndarray, n: int):
    '''Split the dataset samples into `n` equal segments.

    Args:
        data (np.ndarray): Array to be segmented.
        n (int): Number of segments.
    
    Return:
        segmented dataset with shape (batch*`n`, channels, record)
    '''
    # split train data
    seg = np.split(data, n, 2)
    seg = np.stack(seg) # list to array 
    seg = np.moveaxis(seg, 0, 1) # shape [bach, segment, channel, record]

    # merge the `batch` and `segment` dimensions
    dims = seg.shape
    data_segmented = seg.reshape((dims[0]*dims[1], dims[2], dims[3]))

    return data_segmented


def zip_dir(dir_path: str):
    zf = zipfile.ZipFile(dir_path + ".zip", 'w')

    for _, _, files in os.walk(temp_dir.name):
        for filename in files:
            zf.write(temp_dir / filename, filename)

    zf.close()
    shutil.rmtree(temp_dir)

In [17]:
# Slpit the records into 2 seconds segments
# -------------------------------------------------
num_seg = 5

x_train_2sec = segment_samples(x_train, n=num_seg)
y_train_2sec = np.repeat(y_train, num_seg, axis=0)

x_val_2sec = segment_samples(x_val, n=num_seg)
y_val_2sec = np.repeat(y_val, num_seg, axis=0)

x_test_2sec = segment_samples(x_test, n=num_seg)
y_test_2sec = np.repeat(y_test, num_seg, axis=0)

In [18]:
id = 50
sample_info = y_test_df.iloc[id]
print(sample_info)

fig = go.Figure()

fig.add_traces([
    go.Scatter(y=np.concatenate(x_test_2sec[(id*5):(id*5)+5,0,:]), name='concat'),
    go.Scatter(y=x_test[id,0,:], name='original'),
])

fig.update_layout(title=f"ecg_id: {sample_info.ecg_id} {sample_info.label}")
fig.show()

ecg_id        382
strat_fold     10
label          []
CD              0
MI              0
HYP             0
STTC            0
NORM            0
Name: 374, dtype: object


#### Univariate version

In [20]:
x_train_2sec.shape

(87090, 12, 1000)

In [21]:
# Slpit the records into 2 seconds segments
# -------------------------------------------------
if get_univar:
    num_seg = 5
    n_channels = x_test.shape[1]
    sig_len = int(x_test.shape[-1] / num_seg)

    x_train_2sec = segment_samples(x_train, n=num_seg)
    x_train_2sec = x_train_2sec.reshape((-1, sig_len))
    x_train_2sec = zscore(x_train_2sec, axis=1)
    
    y_train_2sec = np.repeat(y_train, num_seg*n_channels, axis=0)

    x_val_2sec = segment_samples(x_val, n=num_seg)
    x_val_2sec = x_val_2sec.reshape((-1, sig_len))
    x_val_2sec = zscore(x_val_2sec, axis=1)

    y_val_2sec = np.repeat(y_val, num_seg*n_channels, axis=0)

    x_test_2sec = segment_samples(x_test, n=num_seg)
    x_test_2sec = x_test_2sec.reshape((-1, sig_len))
    x_test_2sec = zscore(x_test_2sec, axis=1)

    y_test_2sec = np.repeat(y_test, num_seg*n_channels, axis=0)

#### Save the Dataset

In [22]:
print(x_train_2sec.shape, x_val_2sec.shape, x_test_2sec.shape)

# change the dtype to original (float 16)
x_train_2sec = x_train_2sec.astype(np.float16)
x_val_2sec = x_val_2sec.astype(np.float16)
x_test_2sec = x_test_2sec.astype(np.float16)

# check if is normalized to zero mean and unit std
print("mean and std: ", (x_test_2sec[10].mean(), x_test_2sec[10].std()))

((1045080, 1000), (130980, 1000), (131880, 1000))

In [24]:
torch.save({'samples': torch.from_numpy(x_train_2sec), 
            'labels': torch.from_numpy(y_train_2sec)}, 
            f=temp_dir / "train.pt")

torch.save({'samples': torch.from_numpy(x_val_2sec), 
            'labels': torch.from_numpy(y_val_2sec)}, 
            f=temp_dir / "val.pt")

torch.save({'samples': torch.from_numpy(x_test_2sec), 
            'labels': torch.from_numpy(y_test_2sec)}, 
            f=temp_dir / "test.pt")

In [25]:
# read metadata
_, metadata = wfdb.rdsamp(ptbxl_path+"/records500/21000/21000_hr", return_res=16)

# remove channel's information
if x_train_2sec.ndim < 3:
    metadata['n_sig'] = 1
    metadata.pop('units')
    metadata.pop('sig_name')

# length of the full signal
metadata['full_sig_len'] = metadata['sig_len']

# update the signal length
metadata['sig_len'] = x_train_2sec.shape[-1] 

# add the one-hot encode original class name
metadata['labels_code'] = list_label

metadata

{'fs': 500,
 'sig_len': 1000,
 'n_sig': 1,
 'base_date': None,
 'base_time': None,
 'comments': [],
 'full_sig_len': 5000,
 'labels_code': ['CD', 'MI', 'HYP', 'STTC', 'NORM']}

In [34]:
# save the metada file into json format
with open(temp_dir / "metadata.json", 'w') as f:
    json.dump(metadata, f)

# Compact the `temp_dir` into a zip file
zip_dir(temp_dir.name)

## Downsample to 360Hz

In [35]:
import json
import shutil
import zipfile
import torch.nn as nn

In [36]:
dataset_path = Path("../dataset/ptb-xl-2s-univar-700")

with open(dataset_path / "metadata.json", 'r') as f:
    metadata = json.load(f)

train_data = torch.load(dataset_path / 'train.pt')
train_samples = train_data['samples'].unsqueeze(dim=1) # add channel dim

val_data = torch.load(dataset_path / 'val.pt')
val_samples = val_data['samples'].unsqueeze(dim=1) # add channel dim

test_data = torch.load(dataset_path / 'test.pt')
test_samples = test_data['samples'].unsqueeze(dim=1) # add channel dim

In [40]:
new_hz = 360

# compute the new size
duration = (metadata['sig_len']/metadata['fs'])
new_size = int(duration*new_hz)

# interpolate the samples
train_samples = nn.functional.interpolate(train_samples, new_size, mode='linear')
train_samples = train_samples.float()

val_samples = nn.functional.interpolate(val_samples, new_size, mode='linear')
val_samples = val_samples.float()

test_samples = nn.functional.interpolate(test_samples, new_size, mode='linear')
test_samples = test_samples.float()

# update the metadata
metadata['fs'] = new_hz
metadata['full_sig_len'] = int(metadata['full_sig_len']/metadata['sig_len'])*train_samples.shape[-1]
metadata['sig_len'] = train_samples.shape[-1]

In [41]:
fig = go.Figure()

t_original = np.linspace(0, duration, train_data['samples'].shape[-1])
t_resampled = np.linspace(0, duration, train_samples.shape[-1])

fig.add_traces([
    go.Scatter(x=t_original, y=train_data['samples'][0], name='original'),
    go.Scatter(x=t_resampled, y=train_samples[0,0], name='resampled'),
])
fig.update_layout(xaxis_title="Time (s)", width=800, height=300)
fig.show()

In [42]:
temp_dir = Path("ptb-xl-2s-univar-700_360hz")
os.makedirs(temp_dir, exist_ok=True)

# save results in a temporary dicetory
torch.save({'samples': train_samples, 'labels': train_data['labels']}, temp_dir / 'train.pt')
torch.save({'samples': val_samples, 'labels': val_data['labels']}, temp_dir / 'val.pt')
torch.save({'samples': test_samples, 'labels': test_data['labels']}, temp_dir / 'test.pt')

with open(temp_dir / "metadata.json", 'w') as f:
    json.dump(metadata, f)

In [43]:
# zip the temporary directory
zf = zipfile.ZipFile(temp_dir.name + ".zip", 'w')
for dirname, subdir, files in os.walk(temp_dir.name):
    for filename in files:
        zf.write(temp_dir / filename, filename)

zf.close()

shutil.rmtree(temp_dir)